In [ ]:
from langchain.llms import Ollama


import pandas as pd
import subprocess
import glob
import openpyxl

data_folder = "data/Calculator/"
data_files = glob.glob(data_folder + "*.xlsx")
for file in data_files:
    df = pd.read_excel(file)

    sample_data = df.head(5).to_string(index=False)

    prompt = f"""
    I have an Excel file. Here are the first 5 rows (with headers):

    {sample_data}

    Based on this, explain what kind of data this file contains.
    If possible, guess if it is ESG-related (Environment, Social, Governance) or if its a GHG give the scope catagory, 
    financial, HR-related, or something else.
    give me one sentence output.
    """


    ollama = Ollama(
        base_url="http://localhost:11434",
        model="gemma3:1b"
    )

    response = ollama(prompt)
    output_excel = "output_summary.xlsx"
    wb = openpyxl.Workbook()
    ws = wb.active
    ws.title = "Summary"
    ws.append(["Filename", "Response"])
    ws.append([file, response])
    wb.save(output_excel)


This file appears to contain data related to employee numbers and contractor information for a Turkish automotive manufacturing company, potentially including information about female and male employees, and focusing on workforce composition.


In [14]:
###v2

from langchain_community.llms import Ollama
import pandas as pd
import glob
import openpyxl
import os

data_folder = "data/Calculator/"
data_files = glob.glob(os.path.join(data_folder, "*.xlsx"))

# Prepare output workbook
output_excel = "output_summary.xlsx"
if os.path.exists(output_excel):
    wb = openpyxl.load_workbook(output_excel)
    ws = wb.active
else:
    wb = openpyxl.Workbook()
    ws = wb.active
    ws.title = "Summary"
    ws.append(["Filename", "Response"])

# Initialize LLM once
ollama = Ollama(
    base_url="http://localhost:11434",
    model="gemma3:1b"
)

for file in data_files:
    df = pd.read_excel(file)
    sample_data = df.head(5).to_string(index=False)

    prompt = f"""
    Here are the first 5 rows of an Excel file:

    {sample_data}

    In **one short sentence**, explain what kind of data this file contains.
    If it's ESG-related, mention if it's Environment, Social, or Governance.
    If it's GHG-related, mention Scope 1, 2, or 3.
    Otherwise, say Financial, HR-related, or Other.
    """

    response = ollama.invoke(prompt).strip()

    ws.append([os.path.basename(file), response])
    print(f"{file} → {response}")

wb.save(output_excel)
print(f"Summary saved to {output_excel}")




data/Calculator/Scope_3_8_Upstream_Leased_Assets_Asset_Specific_Method.xlsx → This file contains financial data, specifically related to leased asset emissions and energy consumption, covering a period of 2022-23.

It’s an ESG-related dataset because it includes data on greenhouse gas emissions from leased vehicles, specifically focusing on the emission factors of gasoline consumption.
data/Calculator/Scope_3_7_Employee_Commuting_Distance_Based_Method.xlsx → This file contains data about employee commuting distances and related information for various vehicles and organizations, primarily focused on transportation and logistics.

Based on the data, this is a **Financial** file containing data about employee commuting distances and related metrics.
data/Calculator/Scope_1_Direct_Emissions_Mobile_Combustion.xlsx → This file contains financial data, specifically related to vehicle sales and fuel consumption, covering insurance and general insurance activities.
data/Calculator/Scope_3_9_Do

In [15]:
###v3

from langchain_community.llms import Ollama
import pandas as pd
import glob
import openpyxl
import os

data_folder = "data/Calculator/"
data_files = glob.glob(os.path.join(data_folder, "*.xlsx"))

# Prepare output workbook
output_excel = "output_summary.xlsx"
if os.path.exists(output_excel):
    wb = openpyxl.load_workbook(output_excel)
    ws = wb.active
else:
    wb = openpyxl.Workbook()
    ws = wb.active
    ws.title = "Summary"
    ws.append(["Filename", "Response"])

# Initialize LLM once
ollama = Ollama(
    base_url="http://localhost:11434",
    model="gemma3:1b"
    
)

for file in data_files:
    df = pd.read_excel(file)
    sample_data = df.head(5).to_string(index=False)

    prompt = f"""
    Here are the first 5 rows of an Excel file:

    {sample_data}

    Analyze this sample data and determine which GHG Scope it most likely belongs to:
    - Scope 1 (direct emissions from owned/controlled sources),
    - Scope 2 (indirect emissions from purchased energy),
    - Scope 3 (all other value chain emissions).

    If it's not GHG-related, say 'Not GHG-related'.

    Respond in **one short sentence** only.
    """


    response = ollama.invoke(prompt).strip()

    ws.append([os.path.basename(file), response])
    print(f"{file} → {response}")

wb.save(output_excel)
print(f"Summary saved to {output_excel}")




data/Calculator/Scope_3_8_Upstream_Leased_Assets_Asset_Specific_Method.xlsx → Scope_3
data/Calculator/Scope_3_7_Employee_Commuting_Distance_Based_Method.xlsx → Scope_3.7_EmployeeCommuting_DistanceBased
data/Calculator/Scope_1_Direct_Emissions_Mobile_Combustion.xlsx → Scope_1_MobileCombustion
data/Calculator/Scope_3_9_Downstream_Transportation_and_Distribution_Weight_Distance_Based_Method.xlsx → Scope_3
data/Calculator/Scope_3_11_Use_of_Sold_Products_Activity_Based_Method.xlsx → Not GHG-related
data/Calculator/Scope_3_3_Fuel_and_Energy_Related_Activities_Average_Data_Method.xlsx → Scope_3.3_Fuel&EnergyRelatedActivities.
data/Calculator/Scope_1_Direct_Emissions_Stationary_Combustion.xlsx → Scope_1_StationaryCombustion
data/Calculator/Scope_1_Direct_Emissions_Fugitives.xlsx → Scope_1_Fugitives
data/Calculator/Scope_3_6_Business_Travel_Distance_Based_Method.xlsx → Scope_3.6_BusinessTravel_DistanceBased
data/Calculator/Scope_3_1_Purchased_Goods_and_Services_Spend_Based_Method.xlsx → Scope 3

# calculator payload mapping

In [3]:
from langchain_community.llms import Ollama
import pandas as pd

# Define the base payload structure as a template
payload_template = """ 
            invoice = {
                "row_uuid": row.get("uuid"),
                "facility_code": row["Facility Code"],
                "invoice_no": row["Invoice No."],
                "invoice_date": pd.to_datetime(row.get("Invoice Date")).strftime('%Y-%m-%d') if not pd.isnull(row.get("Invoice Date")) else None,
                "start_date": pd.to_datetime(row["Start Date"]).strftime('%Y-%m-%d'),
                "end_date": pd.to_datetime(row["End Date"]).strftime('%Y-%m-%d'),
                "refrigerant_ref_id": row["Refrigerant used"],
                "source_data_unit": row["Unit(s)"],
                "type_of_equipment": row.get("Type of air conditioning and refrigeration equipment"),
                "is_sales_based": is_sales_based,
                "start_inventory": start_inventory,
                "end_inventory": row.get("Refrigerant inventory (in storage, not equipment) at the end of the year", 0),
                "purchased_from_producer": row.get("Refrigerant purchased from producers/distributors", 0),
                "equipment_user_return": row.get("Refrigerant returned by equipment users", 0),
                "recycle_reclamation_return": row.get("Refrigerant returned after off-site recycling or reclamation", 0),
                "delivered_to_equipment": row.get("Refrigerant delivered to equipment users in containers", 0),
                "returned_to_producer": row.get("Refrigerant returned to refrigerant producers", 0),
                "recycle_reclamation_sent_offsite": row.get("Refrigerant sent off-site for recycling or reclamation", 0),
                "destruction_sent_offsite": row.get("Refrigerant sent off-site for destruction", 0),
                "charged_into_equipment": row.get("Refrigerant charged into equipment*", 0),
                "capacity_partial_charged_equipment": row.get("Nameplate Capacity of Partially Charged Equipment", 0),
                "pressure_of_partially_charged": row.get("Density or Pressure** of Partial Charge", 0),
                "pressure_of_fully_charged": row.get("Density or Pressure** of Full Charge", 0),
                "emission_type": row["Activity Data Type"],
                "notes": row.get("Notes"),
                "tag_id": tags if "Tags" in df.columns else [],
                "link": row.get("Evidence"),
                "link_name": row.get("Evidence Name", "Data Source link"),
                "link_note": row.get("Evidence Note"),
                "Updatetype": "System"
            }
"""

# Load Excel data
data_path = "data/Calculator/Scope_1_Direct_Emissions_Fugitives.xlsx"
df = pd.read_excel(data_path)

# Convert only the first few rows to a readable sample for the model
sample_data = df.head(5).to_dict(orient="records")

# Prepare a clean and clear prompt
prompt = f"""
You are a data processing assistant. 

I have an Excel file containing Scope 1 Direct Emissions Fugitives data.  
Below is the payload template for each row:  

{payload_template}

Now, based on the sample data from the Excel file:
{sample_data}

Generate a list of Python dictionaries where each dictionary fills in the above template using the corresponding row values.  
Do not add explanations—just return a valid Python list of dictionaries.
"""

# Initialize Ollama model only once
ollama = Ollama(
    base_url="http://localhost:11434",
    model="gemma3:1b"
)

# Get response from the LLM
response = ollama.invoke(prompt).strip()

print("Generated Payload:\n", response)


Generated Payload:
 ```python
[
    {'GHGCategory': 'Scope_1_Fugitives', 'Organization': 'General Insurance', 'Activity Data Type': 'Actual', 'Facility Name': 'testmulti', 'Facility Code': 'testmulti', 'Invoice No.': 1, 'Invoice Date': Timestamp('2022-01-01 00:00:00'), 'Start Date': Timestamp('2022-01-01 00:00:00'), 'End Date': Timestamp('2022-12-31 00:00:00'), 'Type of air conditioning and refrigeration equipment': 'QA', 'Refrigerant used': 'nitrous oxide', 'Quantity of refrigerant leakage': nan, 'Unit(s)': 'gram (g)', 'Notes': nan, 'Refrigerant inventory (in storage, not equipment) at the beginning of the year': nan, 'Refrigerant inventory (in storage, not equipment) at the end of the year': nan, 'Refrigerant purchased from producers/distributors': nan, 'Refrigerant returned by equipment users': nan, 'Refrigerant returned after off-site recycling or reclamation': nan, 'Nameplate Capacity of Partially Charged Equipment': nan, 'Density or Pressure** of Partial Charge': nan, 'Density or

In [4]:
response


"```python\n[\n    {'GHGCategory': 'Scope_1_Fugitives', 'Organization': 'General Insurance', 'Activity Data Type': 'Actual', 'Facility Name': 'testmulti', 'Facility Code': 'testmulti', 'Invoice No.': 1, 'Invoice Date': Timestamp('2022-01-01 00:00:00'), 'Start Date': Timestamp('2022-01-01 00:00:00'), 'End Date': Timestamp('2022-12-31 00:00:00'), 'Type of air conditioning and refrigeration equipment': 'QA', 'Refrigerant used': 'nitrous oxide', 'Quantity of refrigerant leakage': nan, 'Unit(s)': 'gram (g)', 'Notes': nan, 'Refrigerant inventory (in storage, not equipment) at the beginning of the year': nan, 'Refrigerant inventory (in storage, not equipment) at the end of the year': nan, 'Refrigerant purchased from producers/distributors': nan, 'Refrigerant returned by equipment users': nan, 'Refrigerant returned after off-site recycling or reclamation': nan, 'Nameplate Capacity of Partially Charged Equipment': nan, 'Density or Pressure** of Partial Charge': nan, 'Density or Pressure** of Fu